<a href="https://colab.research.google.com/github/by2lucy/Enjoy-water-sports/blob/main/%EB%AC%B8%EC%A0%9C1_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [ ]:
# CIFAR-10 데이터셋을 불러옵니다.
transf = transforms.Compose([
    transforms.Resize(224),  # 입력 이미지 크기를 224x224로 조정합니다.
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.405), (0.229, 0.224, 0.225))
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transf)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transf)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=32)
testloader = torch.utils.data.DataLoader(testset, batch_size=32)

100%|██████████| 170498071/170498071 [00:03<00:00, 49100857.02it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# VGG16 모델 가져오기
vgg16 = models.vgg16(pretrained=True)

# VGG16 모델의 fully connected layer를 새로운 classifier로 변경합니다.
num_features = vgg16.classifier[6].in_features
vgg16.classifier[6] = torch.nn.Linear(num_features, 10)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# Optimizer와 Loss function 설정
optimizer = torch.optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()

num_epoch = 6
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = vgg16.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(loss)

tensor(2.3885, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.4213, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3202, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2724, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3635, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1952, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1947, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.1003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.0152, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9329, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.0514, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.9767, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.8517, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7132, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.7224, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.6160, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.5649, device='cuda:0', grad_fn=

In [ ]:
# 시도 3: 테스트 성능 측정
correct = 0
total = 0

with torch.no_grad():
    vgg16.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = vgg16(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('테스트 정확도: %.2f %%' % (100 * correct / total))

테스트 정확도: 92.24 %


In [ ]:
# 최종 모델 저장
torch.save(vgg16, '2112876_김여민.pt')

In [ ]:
torch.load('/content/2112876_김여민.pt')

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

시도2

In [ ]:
vgg16_2 = models.vgg16(pretrained=True)

In [ ]:
num_features = vgg16_2.classifier[6].in_features
vgg16_2.classifier[6] = torch.nn.Linear(num_features, 10)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16_2.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# # 시도 2: VGG16 모델에 합성곱 신경망 3개 추가
# conv6 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
# relu6 = torch.nn.ReLU(inplace=True)
# conv7 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
# relu7 = torch.nn.ReLU(inplace=True)
# conv8 = torch.nn.Conv2d(512, 512, kernel_size=3, padding=1)
# relu8 = torch.nn.ReLU(inplace=True)

In [ ]:
# 추가된 합성곱 신경망
vgg16_2.features.add_module('conv6', torch.nn.Conv2d(512, 512, kernel_size=3, padding=1))
vgg16_2.features.add_module('relu6', torch.nn.ReLU(inplace=True))
vgg16_2.features.add_module('conv7', torch.nn.Conv2d(512, 512, kernel_size=3, padding=1))
vgg16_2.features.add_module('relu7', torch.nn.ReLU(inplace=True))
vgg16_2.features.add_module('conv8', torch.nn.Conv2d(512, 512, kernel_size=3, padding=1))
vgg16_2.features.add_module('relu8', torch.nn.ReLU(inplace=True))
vgg16_2.features.add_module('conv9', torch.nn.Conv2d(512, 512, kernel_size=3, padding=1))
vgg16_2.features.add_module('relu9', torch.nn.ReLU(inplace=True))

# 학습을 위해 모델을 GPU로 이동합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16_2.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# 학습을 위해 모델을 GPU로 이동합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16_2.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# 학습 및 평가 부분은 동일하게 진행합니다.
num_epoch = 6
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = vgg16_2.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(loss)

tensor(2.3104, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2724, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2877, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3020, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2857, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2989, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3235, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3227, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3059, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2976, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3177, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3093, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2981, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3065, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3195, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3034, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2921, device='cuda:0', grad_fn=

In [1]:
# 테스트 성능 측정
correct = 0
total = 0

with torch.no_grad():
    vgg16_2.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = vgg16_2(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('시도 2 - 테스트 정확도: %.2f %%' % (100 * correct / total))

NameError: ignored

시도 3

In [ ]:
# VGG16 모델 가져오기
vgg16_3 = models.vgg16(pretrained=True)

# VGG16 모델의 fully connected layer를 새로운 classifier로 변경합니다.
num_features = vgg16_3.classifier[6].in_features
vgg16_3.classifier[6] = torch.nn.Linear(num_features, 10)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16_3.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:01<00:00, 282MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# 추가된 2개의 합성곱 층
vgg16_3.features.add_module('conv6', torch.nn.Conv2d(512, 512, kernel_size=3, padding=1))
vgg16_3.features.add_module('relu6', torch.nn.ReLU(inplace=True))
vgg16_3.features.add_module('conv7', torch.nn.Conv2d(512, 512, kernel_size=3, padding=1))
vgg16_3.features.add_module('relu7', torch.nn.ReLU(inplace=True))

In [ ]:
# 학습을 위해 모델을 GPU로 이동합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16_3.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
# 학습 및 평가 부분은 동일하게 진행합니다.

# Optimizer와 Loss function 설정
optimizer = torch.optim.SGD(vgg16_3.parameters(), lr=0.001, momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()

num_epoch = 8
for i in range(num_epoch):
    for j, [image, label] in enumerate(trainloader):
        x = image.to(device)
        y_ = label.to(device)

        optimizer.zero_grad()
        output = vgg16_3.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

        if i % 10 == 0:
            print(loss)


tensor(2.2871, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2961, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2944, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3042, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2996, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2881, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3120, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3022, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3027, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2908, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2973, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3070, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.2942, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3013, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3146, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3059, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(2.3015, device='cuda:0', grad_fn=

In [ ]:
# 테스트 성능 측정
correct = 0
total = 0

with torch.no_grad():
    vgg16_3.eval()
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = vgg16_3(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('시도 2 - 테스트 정확도: %.2f %%' % (100 * correct / total))


시도 2 - 테스트 정확도: 92.88 %


In [ ]:
# 최종 모델 저장
# torch.save(vgg16_2, '2112876_김여민_2.pt')
torch.save(vgg16_3, '2112876_김여민_3.pt')

# Prob 2: Image Classification with the New Dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import STL10

In [ ]:
# 데이터셋 경로 설정
data_path = './data'

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((56, 56)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [ ]:
# 데이터셋 로드
# split='train' 및 split='test'로 설정하여 학습 및 테스트에 사용할 레이블이 있는 데이터만 로드
train_set = STL10(data_path, split='train', transform=transform, download=True)
test_set = STL10(data_path, split='test', transform=transform, download=True)

# 데이터 로더 생성
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# MyCNN 모델 정의
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(32 * 14 * 14, 10)  # 수정된 선형 레이어

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [ ]:
# 모델 초기화
model = MyCNN().to(device)

# Optimizer와 Loss function 설정
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

# 장치 설정 (GPU 사용 가능 시 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)


MyCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=6272, out_features=10, bias=True)
)

In [ ]:
# 학습 및 평가
num_epochs = 30

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for [images, labels] in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')


Epoch [1/30], Loss: 0.3889, Train Acc: 0.8772
Epoch [2/30], Loss: 0.3623, Train Acc: 0.8872
Epoch [3/30], Loss: 0.3244, Train Acc: 0.9034
Epoch [4/30], Loss: 0.2880, Train Acc: 0.9212
Epoch [5/30], Loss: 0.2754, Train Acc: 0.9182
Epoch [6/30], Loss: 0.2376, Train Acc: 0.9376
Epoch [7/30], Loss: 0.2137, Train Acc: 0.9458
Epoch [8/30], Loss: 0.2007, Train Acc: 0.9518
Epoch [9/30], Loss: 0.1826, Train Acc: 0.9564
Epoch [10/30], Loss: 0.1568, Train Acc: 0.9642
Epoch [11/30], Loss: 0.1457, Train Acc: 0.9708
Epoch [12/30], Loss: 0.1157, Train Acc: 0.9832
Epoch [13/30], Loss: 0.1031, Train Acc: 0.9846
Epoch [14/30], Loss: 0.1033, Train Acc: 0.9840
Epoch [15/30], Loss: 0.0829, Train Acc: 0.9914
Epoch [16/30], Loss: 0.0668, Train Acc: 0.9956
Epoch [17/30], Loss: 0.0602, Train Acc: 0.9968
Epoch [18/30], Loss: 0.0560, Train Acc: 0.9964
Epoch [19/30], Loss: 0.0490, Train Acc: 0.9992
Epoch [20/30], Loss: 0.0474, Train Acc: 0.9988
Epoch [21/30], Loss: 0.0372, Train Acc: 0.9994
Epoch [22/30], Loss: 0

In [ ]:
# 테스트 성능 측정
model.eval()  # 모델을 평가 모드로 설정
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print('테스트 정확도: %.2f %%' % (100 * correct / total))


테스트 정확도: 51.24 %


시도2

In [ ]:
# Device 설정 (GPU 사용 가능 시 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CNN 모델 정의
class CNN3(nn.Module):
    def __init__(self):
        super(CNN3, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc = nn.Linear(64 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

# 모델 초기화
model_3conv = CNN3().to(device)

# Optimizer와 Loss function 설정
optimizer_3conv = optim.SGD(model_3conv.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

# 학습 및 평가
num_epochs = 30

for epoch in range(num_epochs):
    model_3conv.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer_3conv.zero_grad()
        outputs = model_3conv(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer_3conv.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')


Epoch [1/30], Loss: 2.2912, Train Acc: 0.1002
Epoch [2/30], Loss: 2.1958, Train Acc: 0.2038
Epoch [3/30], Loss: 1.9969, Train Acc: 0.2712
Epoch [4/30], Loss: 1.8817, Train Acc: 0.3170
Epoch [5/30], Loss: 1.7759, Train Acc: 0.3520
Epoch [6/30], Loss: 1.6874, Train Acc: 0.3870
Epoch [7/30], Loss: 1.6264, Train Acc: 0.4096
Epoch [8/30], Loss: 1.5676, Train Acc: 0.4330
Epoch [9/30], Loss: 1.5265, Train Acc: 0.4400
Epoch [10/30], Loss: 1.4794, Train Acc: 0.4586
Epoch [11/30], Loss: 1.4306, Train Acc: 0.4786
Epoch [12/30], Loss: 1.3910, Train Acc: 0.4876
Epoch [13/30], Loss: 1.3374, Train Acc: 0.5190
Epoch [14/30], Loss: 1.3074, Train Acc: 0.5318
Epoch [15/30], Loss: 1.2773, Train Acc: 0.5374
Epoch [16/30], Loss: 1.2378, Train Acc: 0.5506
Epoch [17/30], Loss: 1.2097, Train Acc: 0.5674
Epoch [18/30], Loss: 1.1638, Train Acc: 0.5838
Epoch [19/30], Loss: 1.1435, Train Acc: 0.5882
Epoch [20/30], Loss: 1.1108, Train Acc: 0.6082
Epoch [21/30], Loss: 1.0779, Train Acc: 0.6150
Epoch [22/30], Loss: 1

In [ ]:
# 테스트 성능 측정
model_3conv.eval()  # 모델을 평가 모드로 설정
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_3conv(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print('테스트 정확도: %.2f %%' % (100 * correct / total))

  # 20 epoch -> 52.58
  # 30 epoch -> 53.02
  # 35 epoch -> 51.05
  # 50 epoch -> 52.70


테스트 정확도: 51.84 %


시도3

In [ ]:
# Device 설정 (GPU 사용 가능 시 사용)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((56, 56)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


# MyCNN 모델 정의
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.fc = nn.Linear(3 * 56 * 56, 10)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


# 모델 초기화
model_mycnn = MyCNN().to(device)

# Optimizer와 Loss function 설정
optimizer_mycnn = optim.SGD(model_mycnn.parameters(), lr=0.001, momentum=0.9)
loss_func = nn.CrossEntropyLoss()

# 학습 및 평가
num_epochs = 10

for epoch in range(num_epochs):
    model_mycnn.train()  # 모델을 학습 모드로 설정
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer_mycnn.zero_grad()
        outputs = model_mycnn(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer_mycnn.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}')

# 테스트 성능 측정
model_mycnn.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model_mycnn(images)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print('테스트 정확도: %.2f %%' % (100 * correct / total))


Epoch [1/10], Loss: 1.9539, Train Acc: 0.3048
Epoch [2/10], Loss: 1.7676, Train Acc: 0.3834
Epoch [3/10], Loss: 1.6728, Train Acc: 0.4110
Epoch [4/10], Loss: 1.6148, Train Acc: 0.4364
Epoch [5/10], Loss: 1.5386, Train Acc: 0.4692
Epoch [6/10], Loss: 1.5018, Train Acc: 0.4726
Epoch [7/10], Loss: 1.4682, Train Acc: 0.4976
Epoch [8/10], Loss: 1.4197, Train Acc: 0.5158
Epoch [9/10], Loss: 1.3709, Train Acc: 0.5312
Epoch [10/10], Loss: 1.3476, Train Acc: 0.5452
테스트 정확도: 33.71 %
